# Домашнее задание 1. Извлечение ключевых слов

При выполнении домашнего задания можно пользоваться материалами лекций и семинаров.

### Описание задания

1. [X] (1 балл) Подготовить мини-корпус (4-5 текстов или до 10 тысяч токенов) с разметкой ключевых слов. 
Предполагается, что вы найдете источник текстов, в котором **уже выделены** ключевые слова.
Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова. 


2. [X] (2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.
Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).


3. [X] (2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25, ...)


4. [X] (2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun).
Применить эти фильтры к спискам ключевых слов.


5. [X] (2  балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона:
с учётом морфосинтаксических шаблонов и без них.


6. [X] (1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется);
предложить свои методы решения этих проблем.

### Критерии оценки

В каждом пункте указано количество баллов.

### Формат сдачи задания

Jupyter-notebook на гитхабе (запишите адрес своего репозитория [сюда](https://docs.google.com/forms/d/e/1FAIpQLSfvoxiOKm9jnHO6v_ivGOeA3TKBT7Hg7bQlHa56MuALeMIcvQ/viewform?usp=sf_link))

### Дедлайн

9 ноября 2020 23:59мск

## Данные

В качестве данных были выбраны статьи из сборника конференции Dialogue [2019](http://www.dialog-21.ru/digest/2019/articles/) и [2020](http://www.dialog-21.ru/dialogue2020/results/dopmat/2020/scopus/) годов. Я взяла статьи на русском языке и сохранила их название, абстракт, список ключевых слов и заключение (ну и url на всякий случай). Все переносы строк заменила на пробелы просто для удобства хранения.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.tsv', sep='\t')

In [3]:
# data

In [4]:
def count_words(text):
    return len(text.split())

def count_keys(key_words):
    return len(key_words.split(', '))

Получается следующая характеристика корпуса:

In [5]:
data['abs_len'] = data['abstract'].apply(count_words)
data['conc_len'] = data['conclusion'].apply(count_words)
data['len'] = data['abs_len'] + data['conc_len']
print('Всег слов:', data['len'].sum())

data['key_len'] = data['key_words'].apply(count_keys)
data['key_gram'] = data['key_words'].apply(count_words) / data['key_len']

data.describe()

Всег слов: 4178


,abs_len,conc_len,len,key_len,key_gram
count,14.000000,14.000000,14.000000,14.000000,14.000000
mean,112.142857,186.285714,298.428571,5.142857,1.902976
std,44.500401,67.472420,90.597771,1.511858,0.336295
min,37.000000,85.000000,156.000000,3.000000,1.400000
25%,82.750000,142.750000,229.500000,4.000000,1.668750
50%,99.500000,190.500000,295.000000,5.000000,2.000000
75%,154.000000,241.500000,382.000000,6.000000,2.000000
max,179.000000,279.000000,418.000000,8.000000,2.500000


Ключевые слова являются в основном биграммами:

In [6]:
uni = []
bi = []
tri = []

for words in data['key_words']:
    for key in words.split(', '):
        l = len(key.split(' '))
        if l == 1:
            uni.append(key)
        elif l == 2:
            bi.append(key)
        elif l == 3:
            tri.append(key)
        else:
            print(key)

print(f'\nuni {len(uni)}, bi {len(bi)}, tri {len(tri)}')

русский язык в начальной школе

uni 17, bi 46, tri 8


В качестве текста, из которого будут выделяться ключевые слова, я решила использовать склеенные вместе абстракт и заключение.

In [7]:
data['text'] = data['abstract'] + ' | ' + data['conclusion']

In [8]:
# test = data['text'][0]
# print(test)

## Препроцессинг

In [9]:
import re
from razdel import tokenize

from nltk.corpus import stopwords
stops = stopwords.words("russian")
stops.extend(['как', 'который', 'публикация', 'работа', 'это', 'этот'])
for w in ['что', 'то', 'потом']:
    stops.remove(w)
print(stops)

from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
morph = MorphAnalyzer()

from tqdm.auto import tqdm
tqdm.pandas()

['и', 'в', 'во', 'не', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою

In [10]:
def preprocess(text_data):   
    
    tokens = [_.text for _ in list(tokenize(str(text_data).lower()))] # if not re.search('[^а-яА-ЯёЁa-zA-z]', _.text)]
    
    lem_text = []
    for word in tokens:
        lem = morph.parse(word)[0].normal_form
        lem_text.append(lem)
        
    return ' '.join(lem_text)

In [11]:
data['lem'] = data['text'].progress_apply(preprocess)

Заодно нормализуем ключевые слова: лемматизируем и оставим всех по одному

In [12]:
def normalize(key_words):
    lem = preprocess(key_words)
    return ' '.join(sorted(list(set([l for l in lem.split() if l != ',']))))

def make_gold(key_words, my_keys, option='union'):
    if option == 'union':
        full_list = key_words.split()
        full_list.extend(my_keys.split())
    elif option == 'intersect':
        full_list = list(set(key_words.split()) & set(my_keys.split()))
    else:
        print('такой опции нет')
    return ' '.join(sorted(list(set(full_list))))

In [13]:
data['norm_key_words'] = data['key_words'].apply(normalize)
data['norm_my_keys'] = data['my_keys'].apply(normalize)

for opt in ['union', 'intersect']:
    data[f'gold_{opt}'] = data.apply(lambda x: make_gold(x['norm_key_words'],
                                                         x['norm_my_keys'],
                                                         option=opt), axis=1)
    print(data[f'gold_{opt}'].apply(count_words))

0     12
1     12
2     16
3     11
4     12
5      9
6      8
7     15
8     14
9     18
10     6
11    12
12     6
13    11
Name: gold_union, dtype: int64
0     4
1     5
2     3
3     2
4     4
5     6
6     5
7     2
8     3
9     2
10    2
11    2
12    4
13    2
Name: gold_intersect, dtype: int64


## Выделение ключевых слов

In [14]:
import RAKE
from summa import keywords
from gensim.summarization import keywords as kw

In [15]:
rake = RAKE.Rake(stops)

def predict_rake(lem):
    raw_pred = rake.run(lem, maxWords=3, minFrequency=2)
    pred = ', '.join([p[0] for p in raw_pred])
    if len(pred) == 0:
        pred = 'none'
    return pred

In [16]:
def predict_textrank(lem):
    raw_pred = kw(lem, pos_filter=[], scores=True)
    pred = ', '.join([p[0] for p in raw_pred if p[0] not in stops])
    if len(pred) == 0:
        pred = 'none'
    return pred

In [17]:
def predict_summa(lem):
    raw_pred = keywords.keywords(lem, language='russian', additional_stopwords=stops, scores=True)
    pred = ', '.join([p[0] for p in raw_pred])
    if len(pred) == 0:
        pred = 'none'
    return pred

In [18]:
pipeline = {'rake': predict_rake, 
            'textrank': predict_textrank,
            'summa': predict_summa} 

## Морфологические шаблоны

т.к. я нормализую итоговые списки предсказанных ключевых слов, то можно применить к ним простой частеречный фильтр - оставить только те части речи, которые чаще всего встречаются в золотой разметке (`NOUN` и `ADJF`)

In [19]:
from collections import Counter

In [20]:
allowed_pos = Counter()
for key in ' '.join(data['gold_union']).split():
    allowed_pos[morph.parse(key)[0].tag.POS] += 1
    
allowed_pos.most_common()

[('NOUN', 101),
 ('ADJF', 50),
 ('INFN', 4),
 ('PREP', 3),
 ('ADVB', 2),
 (None, 1),
 ('NPRO', 1)]

In [21]:
n_tag = morph.parse('none')[0].tag

def pos_filter(norm_kw):
    res = []
    for kw in norm_kw.split():
        if morph.parse(kw)[0].tag.POS in ['NOUN', 'ADJF']:
            res.append(kw)
    if len(res) == 0:
        res = ['none']
    return ' '.join(res)

## Метрики

Recall = Proportion of relevant words retrieved `(pred ⋂ gold) / gold`

Precision = Proportion of retrieved words that are relevant `(pred ⋂ gold) / pred`

F = `(2*P*R) / (P + R)`

In [22]:
def measure(predicted, golden):
    pred = predicted.split()
    gold = golden.split()
    
    cross = len(list(set(pred) & set(gold)))

    R = cross / len(gold)
    P = cross / len(pred)
    
    if cross == 0:
        F = 0.0
    else:
        F = (2*P*R) / (P + R)
    
    return R, P, F

def mean_metrics(measure):
    df = pd.DataFrame(measure.tolist(), columns=metrics)
    return df.mean()

In [23]:
metrics = ['Recall', 'Precision', 'F-score']

reports = {}
for opt in ['intersect', 'union']:
    reports[opt] = {}
    for filt in ['all', 'filtered']:
        reports[opt][filt] = pd.DataFrame(index=metrics, columns=pipeline.keys())

for method, func in pipeline.items():
    data[f'{method}_pred'] = data['lem'].progress_apply(func)
    data[f'{method}_all'] = data[f'{method}_pred'].apply(normalize)
    data[f'{method}_filtered'] = data[f'{method}_all'].apply(pos_filter)
    
    for opt in ['intersect', 'union']:    
        for filt in ['all', 'filtered']:
            data[f'{method}_{filt}_measure'] = data.apply(lambda x: measure(x[f'{method}_{filt}'], 
                                                                            x[f'gold_{opt}']), axis=1)
            data[f'{method}_{filt}_sum'] = data[f'{method}_{filt}_measure'].apply(sum)

            reports[opt][filt][method] = mean_metrics(data[f'{method}_{filt}_measure'])

In [24]:
reports['union']['all']

,rake,textrank,summa
Recall,0.174937,0.399442,0.484008
Precision,0.339594,0.174556,0.204777
F-score,0.206369,0.237544,0.281830


In [25]:
reports['intersect']['all']

,rake,textrank,summa
Recall,0.316667,0.550000,0.771429
Precision,0.208546,0.078499,0.103025
F-score,0.190548,0.132628,0.175791


In [26]:
reports['union']['filtered']

,rake,textrank,summa
Recall,0.157982,0.352184,0.436750
Precision,0.424079,0.203892,0.247556
F-score,0.203588,0.251074,0.306229


In [27]:
reports['intersect']['filtered']

,rake,textrank,summa
Recall,0.316667,0.523810,0.745238
Precision,0.252176,0.098111,0.130069
F-score,0.214399,0.158479,0.213044


In [28]:
# data

## Анализ ошибок

In [29]:
def missed(method, filt):
    miss = data[data[f'{method}_{filt}_sum'] == 0]
    print(f'полные промахи метода {method}\n')
    
    for row in miss.iterrows():
        print('должно быть:', row[1]['gold_union'])
        print('предсказано:', row[1][f'{method}_{filt}'])
        print('\n')
        
    print(f'итого: {len(miss)} из {len(data)}\n=====')
    return miss

In [30]:
for method in pipeline.keys():
    _ = missed(method, 'all')

полные промахи метода rake

должно быть: интернеткорпус исследование корпус корпусный лингвистика методология нвыка русистика русский технический язык
предсказано: частность что явление


должно быть: в доказательный корпус лексика начальный педагогика русский сложность сравнение текст учебник учебный школа язык
предсказано: none


должно быть: автор авторский атрибуция идентификация классификация текст
предсказано: дать


итого: 3 из 14
=====
полные промахи метода textrank

итого: 0 из 14
=====
полные промахи метода summa

итого: 0 из 14
=====


In [31]:
def analize(method, gold_vocab, filt='all', log=False):
    gold_len = len(gold_vocab)
    
    method_vocab = sorted(list(set(' '.join(data[f'{method}_{filt}']).split())))
    extra = [w for w in method_vocab if w not in gold_vocab]
    missed = [w for w in gold_vocab if w not in method_vocab]
    
    
    if log:
        print(f'метод {method} выделяет лишние слова:\n')
        print(', '.join(extra))
        print('\n')
        print(f'метод {method} НЕ выделяет следующие ключевые слова:\n')
        print(', '.join(missed))
        
        
        print(f'\nитого: {gold_len-len(missed)} из {gold_len} правильных, {len(extra)} слов лишние\n=====')
              
    return extra, missed

Если в качестве стандарта использовать **объединение** ключевых слов, выделенных авторами, и моих:

In [32]:
gold_vocab = sorted(list(set(' '.join(data['gold_union']).split())))
gold_vocab = [g for g in gold_vocab if g not in stops]

for method in pipeline.keys():
    _ = analize(method, gold_vocab, filt='all', log=True)

метод rake выделяет лишние слова:

2, 2,, a, none, выражать, дело, значимость, иван, извиниться, иметь, инвентаризация, использовать, кажется, книга, любовь, метр, например, наш, общественный, однако, орда, отклонение, отсутствие, плат, показать, положение, предложение, проза, разграничительный, разумеется, сат, свидетельствовать, сила, сложный, случай, сравнить, структура, также, талант, то, тоска, тургенев, университет, ф, функция, частность, частотность, число, что, явление, …


метод rake НЕ выделяет следующие ключевые слова:

udapi, автоматический, автор, авторский, анализ, аннотирование, атрибуция, берестяной, близость, валентный, временной, генерация, гикрить, глубинный, говорящий, грамота, деривация, динамик, дискурс, доказательный, достоевский, заполнение, значение, идентификация, интернеткорпус, исследование, классификация, когнитивный, коннектор, корпус, корпусный, лексика, лексический, лингвистика, местоимение, методология, модус, морфосинтаксический, начальный, нвыка, неод

Если применить частеречный фильтр:

In [33]:
gold_vocab = sorted(list(set(' '.join(data['gold_union']).split())))
gold_vocab = [g for g in gold_vocab if g not in stops]

for method in pipeline.keys():
    _ = analize(method, gold_vocab, filt='filtered', log=True)

метод rake выделяет лишние слова:

none, дело, значимость, иван, инвентаризация, книга, любовь, метр, наш, общественный, орда, отклонение, отсутствие, плат, положение, предложение, проза, разграничительный, сила, сложный, случай, структура, талант, тоска, тургенев, университет, функция, частность, частотность, число, явление


метод rake НЕ выделяет следующие ключевые слова:

udapi, автоматический, автор, авторский, анализ, аннотирование, атрибуция, берестяной, близость, валентный, временной, генерация, гикрить, глубинный, говорящий, грамота, дать, деривация, динамик, дискурс, доказательный, достоевский, заполнение, затем, значение, идентификация, интернеткорпус, исследование, классификация, когнитивный, коннектор, корпус, корпусный, лексика, лексический, лингвистика, местоимение, методология, модус, морфосинтаксический, начальный, нвыка, неоднозначный, неопределённый, нкрить, отлклонение, отношение, параллельный, парсинга, педагогика, перевод, переход, повседневный, подход, позиция, п

Если в качестве стандарта использовать **пересечение** ключевых слов, выделенных авторами, и моих:

In [34]:
gold_vocab_intersect = sorted(list(set(' '.join(data['gold_intersect']).split())))
gold_vocab_intersect = [g for g in gold_vocab_intersect if g not in stops]

for method in pipeline.keys():
    _ = analize(method, gold_vocab_intersect, filt='all', log=True)

метод rake выделяет лишние слова:

2, 2,, a, none, валентность, время, выражать, дело, диалог, затем, значимость, иван, извиниться, иметь, инвентаризация, использовать, кажется, класс, книга, любовь, метр, монолог, например, наш, норма, общественный, однако, орда, отклонение, отсутствие, плат, показать, положение, потом, практика, предикат, предложение, проза, разграничительный, разумеется, сат, свидетельствовать, сила, сложный, случай, сравнить, структура, также, талант, то, тоска, тургенев, университет, ф, функция, художественный, частность, частотность, число, что, что-то, эксперимент, электронный, явление, …


метод rake НЕ выделяет следующие ключевые слова:

автоматический, авторский, атрибуция, берестяной, временной, генерация, гикрить, грамота, динамик, достоевский, классификация, корпус, лексический, русский, скетч, следование, содержание, стиль, текст, трансформация, учебник, эпиграфика

итого: 14 из 36 правильных, 65 слов лишние
=====
метод textrank выделяет лишние слова:

ru

### Идеи улучшения:

1. поработать над списком стоп слов (например, сделать кастомое расширение из лексики, специфичной для жанра научной литературы, e.g. *работа, вопрос* - **частично реализовала**
2. задавать параметры анализаторам в зависимости от характеристик конкретного текста
3. удалить из текстов ссылки, лингвистические примеры и не-текстовые токены (это должно убрать лишние именованные сущности и цифры)
4. взять полные тексты статей

Если бы корпус был больше и/или другой тематики (например, новости - в них обычно больше ключевых слов, в т.ч. именованные сущности и группы помимо *Adj+Noun*), то имело бы смысл не делать такую "нормализацию" как у меня, учитывать n-граммы (а не только пословные совпадения) и использовать синтаксические шаблоны.
Прочитав тексты и самостоятельно выделив ключевые слова, я считаю, что для данного корпуса использованные мной методы вполне достаточны, и использование чего-либо более сложного не дало бы особого улучшения.

*Анна Полянская, БКЛ171*